In [2]:
#  http://webgraphviz.com/

In [3]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, pydotplus
%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [4]:
df = pd.read_csv(r'G:\My Drive\Creative_Meta_Analysis\Data\Clean_Data_No_Recode.csv',
                 encoding='latin1', 
                 na_values='', 
                 keep_default_na=False)

In [5]:
df = df[
    (df['Platform'] == 'Desktop') & 
    (df['Region'] == 'NA') & 
    (df['Format'] == 'Video') & 
    (df['Set_Cell'] != 0) &
    ((df['Video_Length'] == ':06') | (df['Video_Length'] == ':15') | (df['Video_Length'] == ':30'))
]

In [6]:
df.shape

(48550, 26)

In [7]:
df.tail()

,Country,Region,Quarter_Year,Campaign,Age_1,Gender,Operating_System,Aided_Awareness,Purchase_Intent,Consideration_Intent,Enjoyment,Interesting,Unique,Involving,Understanding,Relevance,Branding,Brand_Appeal,Distinctiveness,Set_Cell,Creative_Name,Format,Publisher,Platform,Product,Video_Length
265739,US,NA,Q4 2017,US Video Google Home Max Desktop,1.0,0.0,NaN,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,4.0,OOGL3727000H_Home_MaxAnthem_30_MaxOnly_GS_BB_W...,Video,YouTube,Desktop,Google Home Max,:30
265742,US,NA,Q4 2017,US Video Google Home Max Desktop,4.0,0.0,NaN,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,OOGL3727000H_Home_MaxAnthem_30_MaxOnly_GS_BB_W...,Video,YouTube,Desktop,Google Home Max,:30
265746,US,NA,Q4 2017,US Video Google Home Max Desktop,3.0,0.0,NaN,0.0,NaN,0.0,2.0,2.0,2.0,3.0,1.0,1.0,1.0,2.0,1.0,4.0,OOGL3727000H_Home_MaxAnthem_30_MaxOnly_GS_BB_W...,Video,YouTube,Desktop,Google Home Max,:30
265747,US,NA,Q4 2017,US Video Google Home Max Desktop,3.0,0.0,NaN,0.0,NaN,0.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,OOGL3727000H_Home_MaxAnthem_30_MaxOnly_GS_BB_W...,Video,YouTube,Desktop,Google Home Max,:30
265749,US,NA,Q4 2017,US Video Google Home Max Desktop,4.0,0.0,NaN,0.0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,4.0,OOGL3727000H_Home_MaxAnthem_30_MaxOnly_GS_BB_W...,Video,YouTube,Desktop,Google Home Max,:30


### Decision Trees

In [37]:
new_df = df[[
    'Aided_Awareness',
    'Enjoyment',
    'Involving',
    'Interesting',
    'Unique',
    'Understanding',
    'Relevance',
    'Branding',
    'Brand_Appeal',
    'Distinctiveness',
    'Age_1',
    'Video_Length'
]].dropna()

involving = pd.get_dummies(new_df['Involving'], columns=['Involving', 'Mellow', 'Disturbing', 'Weak'])
interesting = pd.get_dummies(new_df['Interesting'], columns=['Interesting', 'Nice', 'Irritating', 'Boring'])
unique = pd.get_dummies(new_df['Unique'], columns=['Unique', 'Soothing', 'Unpleasant', 'Dull'])
vid = pd.get_dummies(new_df['Video_Length'], columns=[':06', ':15', ':30'])

final_df = pd.concat([new_df, involving, interesting, unique, vid], axis=1)

In [41]:
from sklearn.model_selection import train_test_split

X = final_df.drop(['Aided_Awareness', 'Involving', 'Interesting', 'Unique', 'Video_Length'], axis=1)
y = final_df['Aided_Awareness']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import GridSearchCV

dt_aided = DecisionTreeClassifier()
dt_parameters = [{
        'max_depth': [x for x in range(1, 10)]
}]

dt_grid = GridSearchCV(dt_aided, dt_parameters)

In [43]:
dt_train = dt_grid.fit(X_train, y_train)

In [44]:
pd.DataFrame(dt_train.cv_results_)

c:\users\william.raikes\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
c:\users\william.raikes\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
c:\users\william.raikes\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_tra

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_max_depth,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.017366,0.004037,0.734120,0.734120,1,{'max_depth': 1},4,0.734099,0.734130,0.734099,0.734130,0.734160,0.734099,0.002469,0.000018,0.000029,0.000014
1,0.018406,0.004178,0.733565,0.734536,2,{'max_depth': 2},5,0.734099,0.734130,0.732434,0.735379,0.734160,0.734099,0.000253,0.000240,0.000800,0.000596
2,0.022891,0.004524,0.734175,0.734744,3,{'max_depth': 3},3,0.734099,0.734130,0.734266,0.736003,0.734160,0.734099,0.000240,0.000723,0.000069,0.000890
3,0.029089,0.004166,0.734203,0.735119,4,{'max_depth': 4},2,0.733683,0.734504,0.734765,0.736752,0.734160,0.734099,0.001775,0.000245,0.000443,0.001167
4,0.032906,0.003883,0.734231,0.736950,5,{'max_depth': 5},1,0.734932,0.736336,0.733933,0.737918,0.733827,0.736597,0.000254,0.000238,0.000498,0.000692
5,0.037290,0.004180,0.732205,0.738587,6,{'max_depth': 6},6,0.733683,0.738209,0.732018,0.739042,0.730913,0.738511,0.001712,0.000239,0.001138,0.000344
6,0.041597,0.004525,0.730984,0.741543,7,{'max_depth': 7},7,0.731602,0.740873,0.732101,0.741831,0.729248,0.741925,0.001428,0.000017,0.001244,0.000475
7,0.048318,0.004678,0.727265,0.746094,8,{'max_depth': 8},8,0.729604,0.745744,0.729271,0.745369,0.722921,0.747169,0.002648,0.000237,0.003075,0.000776
8,0.049155,0.004680,0.724379,0.751311,9,{'max_depth': 9},9,0.726190,0.750364,0.725524,0.750156,0.721422,0.753413,0.002110,0.000238,0.002108,0.001489


### Visualize Tree

In [45]:
export_graphviz(dt_train.best_estimator_,
                out_file='orig_tree.dot',
                rounded=True,
                feature_names=X_train.columns,
                filled=True,
                class_names=['Unaware', 'Aware'])

### Accuracy Score

In [46]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, dt_train.best_estimator_.predict(X_test))

0.7274947274947275